# 从btckline目录中，导出需要的品种的1分钟K线数据（csv文件)
# 数据来自"蓝目数据" https://www.blueye.info

In [1]:
from datetime import datetime,timedelta
import os,sys,csv,json
import pandas as pd

data_folder = '/home/tensorflow/btckline'

In [2]:
# 导出两个时间段之间得1分钟k线数据,
# start_date  datetime
# end_date    datetime
def export_klines(exchange, symbol_pair,start_date,end_date):    
    
    # 检查开始日期/结束日期格式
    if not isinstance(start_date,datetime): 
        print('use datetime for start_date')
        return
    
    if not isinstance(end_date,datetime):
        print('use datetime for end_date')
        return    
    
    # 计算日期间隔
    days = (end_date - start_date).days
    
    print('from {} to {}, total {} days'.format(start_date, end_date, days))
    
    if days < 1:
        print('not enough days')
        return
    
    bars = []
             
    # 逐一日期，获取数据，合并到bars列表    
    for i in range(days):
        data_date = start_date + timedelta(days = i)
        
        # 根据日期得出目录        
        data_file_folder = os.path.abspath(os.path.join(data_folder,'{}'.format(data_date.strftime('%Y-%m-%d'))))        
        if not os.path.exists(data_file_folder):
            print('{} not exist,pass'.format(data_file_folder),file=sys.stderr)
            continue
        
        # 数据文件名
        data_file_name = os.path.abspath(os.path.join(data_file_folder,'Kline${}&{}&1min.dat'.format(exchange,symbol_pair)))        
        if not os.path.isfile(data_file_name):
            print('{} not exist,pass'.format(data_file_name),file=sys.stderr)
            continue   
        bars_in_day = []
        try:
            # 读取文件，按照json格式加载    
            with open(data_file_name,'r') as f:
                bars_in_day = json.load(f)
                if not isinstance(bars_in_day,list):
                    print('can not load {} into json list'.format(data_file_name))
                    continue
                bars.extend(bars_in_day)
                #print(bars_in_day)
        except Exception as ex:
            print('can not open and load bars from :{},exception:{}'.format(data_file_name,str(ex)))
            continue
    
        print('success load {} bars from:{}'.format(len(bars_in_day), data_file_name))
    
    if len(bars) <1:
        print('no bars from files')
        return
    
    # 加载到dataframe中
    df = pd.DataFrame(bars)
    
    df.columns = ['timestamp','volume','open','close','high','low']
    
    df['datetime']= df['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x/1e3))             
    
    df['date'] = df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df['time'] = df['datetime'].apply(lambda x: x.strftime('%H:%M:%S'))
    
    df=df.set_index('datetime')
    
    df.index.name='index'
    
    
    csv_file_name = os.path.abspath(os.path.join(data_folder,'{}_{}_{}_{}_1m.csv'.format(exchange, symbol_pair, start_date.strftime('%Y%m%d'),end_date.strftime('%Y%m%d'))))
    df.to_csv(csv_file_name,index=True)
    
    print('export bars to:{}'.format(csv_file_name))
                

In [6]:
export_klines(
    exchange='okex',
    symbol_pair= 'bch_usdt',
    start_date = datetime(2018,1,1),
    end_date = datetime(2018,8,31))

from 2018-01-01 00:00:00 to 2018-08-31 00:00:00, total 242 days
success load 1440 bars from:/home/tensorflow/btckline/2018-01-01/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-02/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-03/Kline$okex&bch_usdt&1min.dat
success load 1 bars from:/home/tensorflow/btckline/2018-01-04/Kline$okex&bch_usdt&1min.dat
success load 1117 bars from:/home/tensorflow/btckline/2018-01-05/Kline$okex&bch_usdt&1min.dat
success load 1360 bars from:/home/tensorflow/btckline/2018-01-06/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-07/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-08/Kline$okex&bch_usdt&1min.dat
success load 1404 bars from:/home/tensorflow/btckline/2018-01-09/Kline$okex&bch_usdt&1min.dat
success load 1404 bars from:/home/tensorflow/btckline/2018-01-10/Kline$okex&bch_usdt&1min.dat

/home/tensorflow/btckline/2018-07-25 not exist,pass


success load 1440 bars from:/home/tensorflow/btckline/2018-08-07/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-08/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-09/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-10/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-11/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-12/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-13/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-14/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-15/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-16/Kline$okex&bch_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-0

In [5]:
export_klines(
    exchange='binance',
    symbol_pair= 'bcc_usdt',
    start_date = datetime(2018,1,1),
    end_date = datetime(2018,9,24))

from 2018-01-01 00:00:00 to 2018-09-24 00:00:00, total 266 days
success load 672 bars from:/home/tensorflow/btckline/2018-01-04/Kline$binance&bcc_usdt&1min.dat
success load 702 bars from:/home/tensorflow/btckline/2018-01-05/Kline$binance&bcc_usdt&1min.dat
success load 1360 bars from:/home/tensorflow/btckline/2018-01-06/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-07/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-08/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-09/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-10/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-11/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-12/Kline$binance&bcc_usdt&1min.dat
success load 361 bars from:/home/tensorflow/btckline/2018-01-13/K

/home/tensorflow/btckline/2018-01-01/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-01-02/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-01-03/Kline$binance&bcc_usdt&1min.dat not exist,pass


success load 1439 bars from:/home/tensorflow/btckline/2018-01-19/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-20/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-21/Kline$binance&bcc_usdt&1min.dat
success load 1439 bars from:/home/tensorflow/btckline/2018-01-22/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-23/Kline$binance&bcc_usdt&1min.dat
success load 1438 bars from:/home/tensorflow/btckline/2018-01-24/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-25/Kline$binance&bcc_usdt&1min.dat
success load 1439 bars from:/home/tensorflow/btckline/2018-01-26/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-27/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-01-28/Kline$binance&bcc_usdt&1min.dat
success load 1437 bars from:/h

/home/tensorflow/btckline/2018-07-25 not exist,pass


success load 1440 bars from:/home/tensorflow/btckline/2018-08-05/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-06/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-07/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-08/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-09/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-10/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-11/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-12/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-13/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/home/tensorflow/btckline/2018-08-14/Kline$binance&bcc_usdt&1min.dat
success load 1440 bars from:/h

/home/tensorflow/btckline/2018-09-01/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-09-02/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-09-08/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-09-10/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-09-18/Kline$binance&bcc_usdt&1min.dat not exist,pass
/home/tensorflow/btckline/2018-09-22/Kline$binance&bcc_usdt&1min.dat not exist,pass


export bars to:/home/tensorflow/btckline/binance_bcc_usdt_20180101_20180924_1m.csv
